In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

/home/manish/anaconda2/envs/package15/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [22]:
from scipy.sparse import csr_matrix
from sklearn.model_selection import KFold

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [32]:
train.shape

(4209, 378)

In [33]:
train = train.loc[train['y'] < 250]

In [34]:
train.shape

(4208, 378)

In [3]:
train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [35]:
cols_to_use = [col for col in train.columns if train[col].dtypes == 'object']

In [36]:
cols_to_use.append('y')

In [37]:
ID = 'ID'
TARGET = 'y'

In [38]:
SEED = 0
NFOLDS = 5

In [39]:
xgb_params = {
    'seed': 0,
    'colsample_bytree': 1,
    'silent': 1,
    'subsample': 1.0,
    'learning_rate': 1.0,
    'objective': 'reg:linear',
    'max_depth': 100,
    'num_parallel_tree': 1,
    'min_child_weight': 250,
}

In [40]:
from sklearn.metrics import r2_score

def rsquare_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2_score', r2_score(labels, preds)

In [72]:
cols_to_use.remove('y')

In [73]:
cols_to_use

['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']

In [85]:
def get_data():
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
    
    cols_to_use = [col for col in train.columns if train[col].dtypes == 'object']
    cols_to_use.append('y')
    
    train = train.loc[train[TARGET] < 250]
    train = train[cols_to_use]
    y_train = train[TARGET].ravel()    

    cols_to_use.remove('y')

    test = test[cols_to_use]

    train.drop([TARGET], axis=1, inplace=True)
    #test.drop([ID], axis=1, inplace=True)

    ntrain = train.shape[0]
    train_test = pd.concat((train, test)).reset_index(drop=True)

#     features = train.columns
#     cats = [feat for feat in features if 'cat' in feat]

#     train_test = train_test[cats]

    for feat in train_test.columns:
        train_test[feat] = pd.factorize(train_test[feat], sort=True)[0]

    x_train = np.array(train_test.iloc[:ntrain, :])
    x_test = np.array(train_test.iloc[ntrain:, :])

    return x_train, y_train, x_test


In [51]:
def get_oof(clf, x_train, y_train, x_test):
    ntrain = x_train.shape[0]
    
    oof_train = np.zeros((ntrain,))
    
    kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=SEED).split(x_train)
    
    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]
        
        clf.train(x_train, y_train)
        oof_train[test_index] = clf.predict(x_te)
        
    clf.train(x_train, y_train)
    oof_test = clf.predict(x_test)
    return oof_train.reshape(-1,1), oof_test.reshape(-1,1)
    
    

In [86]:
x_train, y_train, x_test = get_data()

In [88]:
print x_train.shape
print y_train.shape
print x_test.shape

(4208, 8)
(4208,)
(4209, 8)


In [89]:
class XgbWrapper(object):
    def __init__(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds',1000)
        
    def train(self, x_train, y_train,  x_valid=None, y_valid = None, sample_weights=None):
        dtrain = xgb.DMatrix(x_train, label=y_train)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds, feval = rsquare_score)
        
    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x), pred_leaf=True).astype(int)                                                                        

In [90]:
x_train, y_train, x_test = get_data()

In [91]:
dtrain = xgb.DMatrix(x_train, label=y_train)

In [92]:
xg = XgbWrapper(seed=SEED, params=xgb_params)

In [93]:
xg_cat_embedding_train, xg_cat_embedding_test = get_oof(xg, x_train, y_train, x_test)

/home/manish/anaconda2/envs/package15/lib/python2.7/site-packages/ipykernel/__main__.py:14: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.


In [167]:
xg_cat_embedding_train.shape

(4208, 1)

### one hot encoding

In [137]:
cols_to_use = [cols for cols in train.columns if train[cols].dtype == 'object']

In [142]:
bin_train = train[cols_to_use]
bin_test = test[cols_to_use]

In [143]:
bin_all = pd.concat([bin_train, bin_test])

In [145]:
bin_all.shape

(8417, 8)

In [146]:
del dummie_train

In [147]:
dummie_all = pd.get_dummies(bin_all)

In [148]:
dummie_all

,X0_a,X0_aa,X0_ab,X0_ac,X0_ad,X0_ae,X0_af,X0_ag,X0_ai,X0_aj,...,X8_p,X8_q,X8_r,X8_s,X8_t,X8_u,X8_v,X8_w,X8_x,X8_y
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [150]:
x_train_ohe = dummie_all[:train.shape[0]]
x_test_ohe = dummie_all[train.shape[0]:]

In [152]:
print x_train_ohe.shape
print x_test_ohe.shape

(4208, 211)
(4209, 211)


In [156]:
type(x_train_ohe)

pandas.core.frame.DataFrame

In [161]:
xg_onehot_embed_train, xg_onehot_embed_test = get_oof(xg, x_train=x_train_ohe.as_matrix(), y_train=y_train, x_test=x_test_ohe.as_matrix())

/home/manish/anaconda2/envs/package15/lib/python2.7/site-packages/ipykernel/__main__.py:14: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.


In [163]:
xg_onehot_embed_train

array([[  9.],
       [  9.],
       [  9.],
       ..., 
       [ 23.],
       [  4.],
       [ 23.]])

### create features

#### integer encoding

In [168]:
type(xg_cat_embedding_train)

numpy.ndarray

In [169]:
train_dim = pd.DataFrame(xg_cat_embedding_train.tolist())

In [182]:
train_dim.columns = ['integer_embedding']

In [185]:
train_dim['ID'] = train.ID

In [187]:
test_dim = pd.DataFrame(xg_cat_embedding_test.tolist())
test_dim.columns = ['integer_embedding']
test_dim['ID'] = test.ID

In [189]:
train_dim.to_csv("features/train_integer_embedding.csv", index=False)
test_dim.to_csv("features/test_integer_embedding.csv", index=False)

#### onehot encoding

In [190]:
train_dim = pd.DataFrame(xg_onehot_embed_train.tolist())
test_dim = pd.DataFrame(xg_onehot_embed_test.tolist())

In [191]:
train_dim.columns = ['onehot_embedding']
test_dim.columns = ['onehot_embedding']
train_dim['ID'] = train.ID
test_dim['ID'] = test.ID

In [193]:
train_dim.to_csv("features/train_onehot_embedding.csv")
test_dim.to_csv("features/test_onehot_embedding.csv")